In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/dataset1')

In [ ]:
!ls

 good   hole   objects	'oil spot'  'thread error'


In [ ]:
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Define data transformations for preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
dataset = ImageFolder(root="/content/drive/MyDrive/dataset1", transform=transform)

# Create data loader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the class names
class_names = dataset.classes


## CNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the CNN model
num_classes = len(class_names)
cnn_model = SimpleCNN(num_classes)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_model.parameters(), lr=0.005, momentum=0.9)

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    cnn_model.train()  # Set the model to training mode

    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {loss:.4f}")

# Evaluation code
def evaluate_model(model):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad ():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

# Evaluate the trained CNN model
evaluate_model(cnn_model)


Epoch [1/2] - Loss: 1.0996
Epoch [2/2] - Loss: 1.0452
Accuracy: 55.47%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
# Define a simple CNN model-4
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)  #reshape tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the CNN model
num_classes = len(class_names)
cnn_model = SimpleCNN(num_classes)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss() #loss function
optimizer = optim.SGD(cnn_model.parameters(), lr=0.005, momentum=0.9)   #update the model's parameters during training to minimize the loss.

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    cnn_model.train()  # Set the model to training mode

    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {loss:.4f}")

# Evaluation code
def evaluate_model(model):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad ():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

# Evaluate the trained CNN model
evaluate_model(cnn_model)


Epoch [1/5] - Loss: 1.1838
Epoch [2/5] - Loss: 1.4479
Epoch [3/5] - Loss: 1.2063
Epoch [4/5] - Loss: 1.0560
Epoch [5/5] - Loss: 0.7367
Accuracy: 61.10%


## VGG 19

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# Initialize the VGG19 model with pre-trained weights
vgg19 = models.vgg19(pretrained=True)

# Modify the last fully connected layer to match the number of classes in your dataset
num_classes = len(class_names)
vgg19.classifier[6] = nn.Linear(vgg19.classifier[6].in_features, num_classes)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg19.parameters(), lr=0.005, momentum=0.9)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    vgg19.train()  # Set the model to training mode

    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = vgg19(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {loss:.4f}")

# Evaluation code
def evaluate_model(model):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

# Evaluate the trained VGG19 model
evaluate_model(vgg19)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:06<00:00, 82.4MB/s]


KeyboardInterrupt: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# Initialize the ResNet-34 model with pre-trained weights
resnet34 = models.resnet34(pretrained=True)

# Modify the last fully connected layer to match the number of classes in your dataset
num_classes = len(class_names)
resnet34.fc = nn.Linear(resnet34.fc.in_features, num_classes)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet34.parameters(), lr=0.010, momentum=0.9)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    resnet34.train()  # Set the model to training mode

    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = resnet34(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {loss:.4f}")

# Evaluation code
def evaluate_model(model):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

# Evaluate the trained ResNet-34 model
evaluate_model(resnet34)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 131MB/s]


##ResNet-34

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# Initialize the ResNet-34 model with pre-trained weights
resnet34 = models.resnet34(pretrained=True)

# Modify the last fully connected layer to match the number of classes in your dataset
num_classes = len(class_names)
resnet34.fc = nn.Linear(resnet34.fc.in_features, num_classes)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet34.parameters(), lr=0.010, momentum=0.9)

# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    resnet34.train()  # Set the model to training mode

    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = resnet34(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {loss:.4f}")

# Evaluation code
def evaluate_model(model):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

# Evaluate the trained ResNet-34 model
evaluate_model(resnet34)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 62.3MB/s]


Epoch [1/1] - Loss: 0.6742
Accuracy: 64.94%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms, datasets

# Load your dataset and define the dataloader
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Replace '/path/to/your/dataset' with the actual path to your dataset
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/dataset1', transform=data_transforms)
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize the ResNet-34 model with pre-trained weights
resnet34 = models.resnet34(pretrained=True)

# Modify the last fully connected layer to match the number of classes in your dataset
num_classes = len(train_dataset.classes)
resnet34.fc = nn.Linear(resnet34.fc.in_features, num_classes)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()  #train a model to predict one class among multiple possible classes
optimizer = optim.SGD(resnet34.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    resnet34.train()  # Set the model to training mode

    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = resnet34(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {loss:.4f}")

# Evaluation code
def evaluate_model(model):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

# Evaluate the trained ResNet-34 model
evaluate_model(resnet34)


Epoch [1/3] - Loss: 0.6097
Epoch [2/3] - Loss: 0.4116
Epoch [3/3] - Loss: 0.4517
Accuracy: 83.95%
